In [46]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items)) 
        
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 리스트를 생성합니다. 값은 콤마로만 구분해주세요."),
    ("system", "리스트 크기는 {max_length}를 넘을 수 없고, 값은 중복될 수 없습니다."),
    ("human", "{question}")
])

# prompt = template.format_messages(max_length=10, question="국가들을 알려주세요." )
# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_length":5,
    "question":"태국의 도시들을 알려주세요."
})

['방콕(8)', '치앙마이(1)', '푸켓(0.4)', '파타야(0.4)', '나컨랏치사마(0.3)']